In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image, ImageDraw

In [2]:
data = pd.read_csv('10fold/pneumonia-test-0.csv', sep=';')
data2 = pd.read_csv('10fold/pneumonia-train-0.csv', sep=';')
data = data.append(data2, ignore_index=True)
print(data['Target'].value_counts())
data_empty = data.loc[data['Target'] == 0]
data = data.loc[data['Target'] != 0]
data.reset_index(drop=True)
data['center_x'] = data['x'] + data['width'] / 2
data['center_y'] = data['y'] + data['height'] / 2
print(len(data))
data.describe()

0    20672
1     9555
Name: Target, dtype: int64
9555


,x,y,width,height,Target,center_x,center_y
count,9555.000000,9555.000000,9555.000000,9555.000000,9555.0,9555.000000,9555.000000
mean,394.047724,366.839560,218.471376,329.269702,1.0,503.283412,531.474411
std,204.574172,148.940488,59.289475,157.750755,0.0,204.982559,115.071015
min,2.000000,2.000000,40.000000,45.000000,1.0,87.500000,130.000000
25%,207.000000,249.000000,177.000000,203.000000,1.0,314.500000,455.000000
50%,324.000000,365.000000,217.000000,298.000000,1.0,415.000000,531.000000
75%,594.000000,478.500000,259.000000,438.000000,1.0,709.000000,609.000000
max,835.000000,881.000000,528.000000,942.000000,1.0,899.500000,945.000000


In [5]:
data['min_x'] = data['x']
data['min_y'] = data['y']
data['max_x'] = data['x'] + data['width']
data['max_y'] = data['y'] + data['height']
total_examples = len(data)

for ratio in range(5, 10, 1):
    ratio_column = f'inside {ratio/10}'
    length_ratio = (ratio/10) ** 0.5
    bbox_size = int(length_ratio * 1024)
    border_size = (1024 - bbox_size) / 2
    bbox_start = border_size
    bbox_end = 1024 - border_size
    inside_examples = 0
    inside_column = np.zeros(total_examples)
    
    for k in range(total_examples):
        example = data.iloc[k].to_dict()  # type: dict
        if example['min_x'] > bbox_start and example['max_x'] < bbox_end:
            if example['min_y'] > bbox_start and example['max_y'] < bbox_end:
                inside_examples += 1
                inside_column[k] = 1.0
    data[ratio_column] = inside_column
    print(ratio/10, inside_examples/total_examples, total_examples - inside_examples)
data.describe()

0.5 0.696493982208268 2900
0.6 0.8458398744113029 1473
0.7 0.9290423861852434 678
0.8 0.9721611721611721 266
0.9 0.9899529042386185 96


,x,y,width,height,Target,center_x,center_y,min_x,min_y,max_x,max_y,inside 0.5,inside 0.6,inside 0.7,inside 0.8,inside 0.9
count,9555.000000,9555.000000,9555.000000,9555.000000,9555.0,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000,9555.000000
mean,394.047724,366.839560,218.471376,329.269702,1.0,503.283412,531.474411,394.047724,366.839560,612.519100,696.109262,0.696494,0.845840,0.929042,0.972161,0.989953
std,204.574172,148.940488,59.289475,157.750755,0.0,204.982559,115.071015,204.574172,148.940488,209.625208,129.391106,0.459796,0.361121,0.256767,0.164519,0.099736
min,2.000000,2.000000,40.000000,45.000000,1.0,87.500000,130.000000,2.000000,2.000000,147.000000,214.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,207.000000,249.000000,177.000000,203.000000,1.0,314.500000,455.000000,207.000000,249.000000,423.500000,614.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,324.000000,365.000000,217.000000,298.000000,1.0,415.000000,531.000000,324.000000,365.000000,519.000000,705.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,594.000000,478.500000,259.000000,438.000000,1.0,709.000000,609.000000,594.000000,478.500000,821.000000,785.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,835.000000,881.000000,528.000000,942.000000,1.0,899.500000,945.000000,835.000000,881.000000,1024.000000,1024.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
print(len(data))
for ratio in range(5, 10, 1):
    ratio_column = f'inside {ratio/10}'
    print(data[ratio_column].value_counts())
    

9555
1.0    6655
0.0    2900
Name: inside 0.5, dtype: int64
1.0    8082
0.0    1473
Name: inside 0.6, dtype: int64
1.0    8877
0.0     678
Name: inside 0.7, dtype: int64
1.0    9289
0.0     266
Name: inside 0.8, dtype: int64
1.0    9459
0.0      96
Name: inside 0.9, dtype: int64


In [9]:
# border data
for ratio in range(5, 10, 1):
    ratio_column = f'inside {ratio/10}'
    ratio_data = data.loc[data[ratio_column] == 1.0]
    #print(ratio_data)
    ratio_data = ratio_data.append(data_empty, ignore_index=True)
    #print(ratio_data)
    print(ratio_data['Target'].value_counts())
    ratio_data.to_csv(f'labels_inside_{ratio/10}.csv', index=False, encoding='utf-8')

0    20672
1     6655
Name: Target, dtype: int64
0    20672
1     8082
Name: Target, dtype: int64
0    20672
1     8877
Name: Target, dtype: int64
0    20672
1     9289
Name: Target, dtype: int64
0    20672
1     9459
Name: Target, dtype: int64


In [23]:
# negative samples
patients = []
for patientId in data_empty['patientId'][:20]:
    print(f"\"{patientId}\"", end=' ')
    patients.append(patientId)
print()
print(patients)

# positive samples
patients = []
for patientId in data['patientId'].drop_duplicates()[:20]:
    print(f"\"{patientId}\"", end=' ')
    patients.append(patientId)
patients

In [15]:
repeated = data.copy()
repeated = repeated.loc[repeated.duplicated(subset='patientId', keep=False)].reset_index(drop=True)
patients = []
num_repetitions = []
x_center = []
x_min = []
x_max = []
y_center = []
y_min = []
y_max = []
for k in range(len(repeated)):
    entry = repeated.iloc[k].to_dict()
    if k == 0 or patients[-1] != entry['patientId']:
        x_min.append(entry['x'])
        y_min.append(entry['y'])
        x_max.append(entry['x'] + entry['width'])
        y_max.append(entry['y'] + entry['height'])
        num_repetitions.append(1)
        x_center.append(entry['center_x'])
        y_center.append(entry['center_y'])
        patients.append(entry['patientId'])
    else:
        num_repetitions[-1] += 1
        if x_min[-1] > entry['x']:
            x_min[-1] = entry['x']
        if y_min[-1] > entry['y']:
            y_min[-1] = entry['y']
        if x_max[-1] < entry['x'] + entry['width']:
            x_max[-1] = entry['x'] + entry['width']
        if y_max[-1] < entry['y'] + entry['height']:
            y_max[-1] = entry['y'] + entry['height']

repeated = pd.DataFrame.from_dict({'patientId': patients,
                         'x_min': x_min,
                         'x_max': x_max,
                         'y_min': y_min,
                         'y_max': y_max,
                         })
repeated['center_x'] = (repeated['x_max'] + repeated['x_min']) / 2
repeated['center_y'] = (repeated['y_max'] + repeated['y_min']) / 2
repeated['width'] = repeated['x_max'] - repeated['x_min']
repeated['height'] = repeated['y_max'] - repeated['y_min']
repeated.describe()

In [16]:
unique = data.loc[~data.duplicated(subset='patientId', keep=False)].reset_index(drop=True)
unique['min_x'] = unique['x']
unique['min_y'] = unique['y']
unique['center_x'] = unique['min_x'] + unique['width'] / 2
unique['center_y'] = unique['min_y'] + unique['height'] / 2
unique.describe()

In [17]:
from sklearn.mixture import GaussianMixture
def double_gaussian(dataframe_column):
    return GaussianMixture(n_components=2).fit(dataframe_column.to_numpy().reshape(-1, 1))

def display_double_gaussian(distribution):
    means_hat = distribution.means_.flatten()
    weights_hat = distribution.weights_.flatten()
    sds_hat = np.sqrt(distribution.covariances_).flatten()
    
    print(distribution.converged_)
    print(means_hat)
    print(sds_hat)
    print(weights_hat)
    
    base = np.linspace(0, 1024, 100)
    return plt.plot(base, np.exp(distribution.score_samples(base[:, np.newaxis])))


In [18]:
mixture = double_gaussian(data['x'])
display_double_gaussian(mixture)

In [19]:
center_x_dist = double_gaussian(data['center_x'])
display_double_gaussian(mixture)

In [7]:
def draw_rectangle(image, x_min, y_min, width, height, color='yellow', size=1):
    draw = ImageDraw.Draw(image)
    coordinates = [x_min, y_min, x_min + width + 1, y_min + height + 1]
    draw.rectangle(xy=coordinates, width=size, outline=color)
    return image

def show_grid(images, images_per_line=3):
    #creates a new empty image, RGB mode, and size 400 by 400.
    width, height = images[0].size
    num_lines = int(np.ceil(len(images) / images_per_line))
    grid_width = int(width * images_per_line)
    grid_height = int(height * num_lines)
    background = Image.new('RGB', (grid_width, grid_height))
    
    #Iterate through a 4 by 4 grid with 100 spacing, to place my image
    for i in range(num_lines):
        for j in range(images_per_line):
            idx = i * images_per_line + j
            image = images[idx]
            x = j * width
            y = i * height
            #paste the image at location i,j:
            background.paste(image, (x,y))
    
    return background

In [8]:
num_images = 10
names = []
for i in range(num_images):
    data_filtered = data.drop_duplicates(subset='patientId')
    example = data_filtered.iloc[i]
    patientId = example['patientId']
    name = patientId + '.jpg'
    print(name)
    names.append(name)

images = []
for i in range(num_images):
    name = names[i]
    examples = data.loc[data['patientId'] == name[:-4]]
    
    image = Image.open(os.path.join('images/train', name))
    
    for j in range(len(examples)):
        example = examples.iloc[j]
        image = draw_rectangle(image=image,
                                 x_min=example['x'],
                                 y_min=example['y'],
                                 width=example['width'],
                                 height=example['height'],
                                 color='yellow',
                                 size=4)
    images.append(image)
# show_grid(images, images_per_line=2)